# Análise Comparativa

O objetivo dessa etapa é comparar modelos para encontrar um que possa ser melhor utilizado dentro do problema em questão (detecção doença cardiovascular), para isso antes realizamos a preparação e o pré-processamento dos dados.

## 1. Preparação dos dados


### 1.1 Configurações iniciais
Importações e configurações

In [16]:
from pathlib import Path
import joblib
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

### 1.2 Obtendo dados
Nessa etapa obtemos novamnete os arquivos brutos de dados e o dicionário antes de começarmos o pre-processamento.

In [3]:
dataset_path = Path('../data/raw/dataset.csv')
dict_path = Path('../data/external/dicionario.csv')

In [4]:
df = pd.read_csv(dataset_path)
df_dict = pd.read_csv(dict_path)

### 1.3 Tratamento de dados
Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados

In [5]:
target_column = 'target'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

nominal_one_hot_columns = ['chest pain type', 'resting ecg', 'ST slope']

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [ ]:
one_hot_preprocessor = Pipeline([
    ('encoding', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
    ('normalization', MinMaxScaler()) # normalização de dados

])

nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('normalization', MinMaxScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', MinMaxScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('one_hot', one_hot_preprocessor, nominal_one_hot_columns),
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

## 2. Escolha do modelo

### 2.1 Metodologia
Iremos análisar seis modelos, que serão testados utilizando um método de validação cruzada por permutação, os modelos que serão testados serão: 

- Logistic Regression (LR)
- K-Nearest-Neighbors (KNN)
- MultiLayer Perceptron (MLP)
- Gaussian Naive Bayes (NB)
- Decision Trees (DT)
- Random Forest (RF)

Além disso, cada um desses algoritmos será testado com diferentes parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Os modelos serão comparados através dos seguintes parâmetros
- `accuracy`: Proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados

- `precision` : Proporção entre dados corretamente previstos como positivos e o total de observações positivas. Ou seja, de todos os pacientes que identificamos com doença do coração, quantos realmente possuem?

- `recall`: Proporção entre dados corretamente previstos como positivos com o total de observações. Ou seja, entre todos os pacientes que possuem doença do coração, quantos foram marcados? Em outras palavras, esse parâmetro ajuda a identificar a sensibilidade do nosso modelo

- `f1`: Média ponderada entre `precision` e `recall`, portanto levando em conta tanto falsos positivos quanto falsos negativos.

In [19]:
params_knn = {
    'n_neighbors': range(3, 20, 2),
    'weights': ['uniform', 'distance']}


params_logr = {
    'C' : np.logspace(-4, 4, 20),
}


params_nb = {'var_smoothing' : [10**(-9)]}

params_svm = {"kernel": ["linear", "rbf"], 'C':[1,10,100,1000],'gamma':[0.0001, 0.001, 0.1, 1]}

params_mlp = {
#     'hidden_layer_sizes': [(200,50, 30), (100,50, 10),(100,50),(200,100),(500,250), (20,), (50,), (100,), (10,), (200,)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001, 0.005, 0.05],
#     'learning_rate': ['constant', 'adaptive']
    
}
params_rfc = {    
#     'n_estimators': [100, 300, 500, 800, 1200],
#     'max_depth': [5, 8, 15, 25, 30],
#     'min_samples_split': [2, 5, 10, 15, 100],
#     'min_samples_leaf': [1, 2, 5, 10],
#     'max_features': [1, 2, 3, 4, 5]
            }

params_dtc =  {
    'criterion':['gini','entropy'],
    'max_depth': [3, 6, 8]
}

### 2.2 Configuração do experimento
Iremos definir as configurações dos modelos, separar o conjunto de dados em conjunto de testes e de treino para realizar a validação cruzada alem de realizar o teste com os modelos definidos anteriormente, testando também os diferentes parâmetros para obter a melhor combinação possível. 

In [8]:
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
random_state = 42
# model settings
models = [
    ('KNN', KNeighborsClassifier(), params_knn),
    ('Logistic Regression', LogisticRegression(), params_logr),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), params_dtc),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), params_rfc),
    ('Gaussian Naive Bayes', GaussianNB(), params_nb),
    ('Multilayer Perceptron', MLPClassifier(random_state=random_state), params_mlp)
]

In [9]:
cv = ShuffleSplit(n_splits=30, train_size=0.8, random_state=42)


results = {}
for model_name, model, model_params in models:
    print(f'{model_name} run...')
    
    model_gs = GridSearchCV(model, model_params, scoring=scoring)
    approach = Pipeline([
        ("preprocessing", preprocessor),
        ("model", model_gs)
    ])
    model_results = cross_validate(
        approach,
        X=X,
        y=y,
        scoring=metrics,
        cv=cv,
        n_jobs=-1,
        return_train_score=False
    )
    model_results['name'] = [model_name] * len(model_results['score_time'])
    if results:
        for key, value in model_results.items():
            results[key] = np.append(results[key], value)
    else:
        results = model_results

KNN run...
Logistic Regression run...
Decision Tree run...
Random Forest run...
Gaussian Naive Bayes run...
Multilayer Perceptron run...


### 2.3 Resultados
Aqui iremos comparar os resultados e definir um modelo para ser utilizado no projeto

In [10]:
df_results = pd.DataFrame(results)
df_results.groupby('name').agg([np.mean, np.std])

fit_time           score_time           test_accuracy  \
                            mean       std       mean       std          mean   
name                                                                            
Decision Tree           0.855871  0.307269   0.069075  0.030115      0.833613   
Gaussian Naive Bayes    0.245785  0.081266   0.098910  0.045042      0.743557   
KNN                    14.581691  2.766774   0.181954  0.065873      0.852241   
Logistic Regression     4.159901  0.786623   0.060665  0.020251      0.839776   
Multilayer Perceptron  22.281252  2.347652   0.111250  0.066030      0.847899   
Random Forest           8.765935  2.243916   0.174466  0.089114      0.856443   

                                test_precision_macro            \
                            std                 mean       std   
name                                                             
Decision Tree          0.027154             0.834211  0.026828   
Gaussian Naive Bayes   0.040127             0.786759  0.026411   
KNN                    0.028520             0.851887  0.028638   
Logistic Regression    0.025324             0.840622  0.025662   
Multilayer Perceptron  0.022299             0.848662  0.022681   
Random Forest          0.022399             0.855955  0.022802   

                      test_recall_macro           test_f1_macro            
                                   mean       std          mean       std  
name                                                                       
Decision Tree                  0.833175  0.027765      0.832442  0.027447  
Gaussian Naive Bayes           0.753441  0.034469      0.736912  0.041876  
KNN                            0.852949  0.029075      0.851591  0.028724  
Logistic Regression            0.837810  0.024894      0.838266  0.025185  
Multilayer Perceptron          0.846306  0.021731      0.846576  0.022102  
Random Forest                  0.856295  0.022305      0.855632  0.022549

In [20]:
#Cria uma tabela para melhor visualizar e comparar os modelos 

def highlight_max(s, props=''):
    values = [float(value.split()[0]) for value in s.values[1:]]
    result = [''] * len(s.values)
    if s.values[0].endswith('time'):
        result[np.argmin(values)+1] = props
    else:
        result[np.argmax(values)+1] = props
    return result

def get_winner(s):
    metric = s.values[0]
    values = [float(value.split()[0]) for value in s.values[1:]]
    models = results.columns[1:]
    
    if s.values[0].endswith('time'):
        return models[np.argmin(values)]
    else:
        return models[np.argmax(values)]

results = (
    pd
    .DataFrame(df_results)
    .groupby(['name'])
    .agg([lambda x: f"{np.mean(x):.3f} ± {np.std(x):.3f}"])#
    .transpose()
    .reset_index()
    .rename(columns={"level_0": "score"})
    .drop(columns="level_1")
    # .set_index('score')
)
time_scores = ['fit_time', 'score_time']
winner = results.query('score not in @time_scores').apply(get_winner, axis=1).value_counts().index[0]
results.columns.name = ''
results = (
    results
    .style
    .hide(axis='index')
    .apply(highlight_max, props='color:white;background-color:gray', axis=1)
)
display(results)
display(Markdown(f'### O melhor modelo é o : **{winner}**'))

score,Decision Tree,Gaussian Naive Bayes,KNN,Logistic Regression,Multilayer Perceptron,Random Forest
fit_time,0.856 ± 0.302,0.246 ± 0.080,14.582 ± 2.720,4.160 ± 0.773,22.281 ± 2.308,8.766 ± 2.206
score_time,0.069 ± 0.030,0.099 ± 0.044,0.182 ± 0.065,0.061 ± 0.020,0.111 ± 0.065,0.174 ± 0.088
test_accuracy,0.834 ± 0.027,0.744 ± 0.039,0.852 ± 0.028,0.840 ± 0.025,0.848 ± 0.022,0.856 ± 0.022
test_precision_macro,0.834 ± 0.026,0.787 ± 0.026,0.852 ± 0.028,0.841 ± 0.025,0.849 ± 0.022,0.856 ± 0.022
test_recall_macro,0.833 ± 0.027,0.753 ± 0.034,0.853 ± 0.029,0.838 ± 0.024,0.846 ± 0.021,0.856 ± 0.022
test_f1_macro,0.832 ± 0.027,0.737 ± 0.041,0.852 ± 0.028,0.838 ± 0.025,0.847 ± 0.022,0.856 ± 0.022


### O melhor modelo é o : **Random Forest**

### 2.5 Persistência do modelo
Com isso, definimos que o melhor modelo é o de **Random Forest**, portanto podemos obter os melhores parâmetros desse modelo e salvar esse modelo em disco para utilizar na pŕóxima fase da análise

In [21]:
#Obtem o modelo e os parametros ganhadores
model_name, model, model_params  = [foo for foo in models if foo[0] == winner][0] 

model_gs = GridSearchCV(model, model_params, scoring='accuracy')
approach = Pipeline([
        ("preprocessing", preprocessor),
        ("model", model_gs)
])
approach.fit(X, y) #Seleciona o approach

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('one_hot',
                                                  Pipeline(steps=[('encoding',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False)),
                                                                  ('normalization',
                                                                   MinMaxScaler())]),
                                                  ['chest pain type',
                                                   'resting ecg', 'ST slope']),
                                                 ('nominal',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('normalization',
                                                                   MinMaxScaler())]),
                                                  ['sex', 'chest pain type',
                                                   'fasting blood sugar',
                                                   'resting ecg',
                                                   'exercise angina',
                                                   'ST slope']),
                                                 ('continuous',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer()),
                                                                  ('normalization',
                                                                   MinMaxScaler())]),
                                                  ['oldpeak'])])),
                ('model',
                 GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                              param_grid={}, scoring='accuracy'))])

In [17]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco

['../models/model.joblib']